In [20]:
import pandas as pd
import numpy as np
import collections #统计
import nltk #自然语言处理包
from nltk.collocations import * 
from gensim.models import word2vec #词向量
import re

### 以句子为单位分

In [26]:
dynasty=''
path = r'C:/Users/HP/新建文件夹/pa_chong/poem/朝代/唐代total.csv'
f=open(path,'rb')
content = pd.read_csv(f, header=None)[1:]
f.close()
poems=content[3]

In [27]:
def make_sentence(poem):
    sentence = [i for i in re.split('[。?!]', poem) if len(i)>1]
    return sentence

In [34]:
pos=1
length= len(poems)


In [35]:
sentence=[]

In [36]:
try:
    while pos<=length:
        words=[]
        try:
            for i in range(1,length+1):
                words=words+make_sentence(poems.get(pos))
                pos=pos+1
                if(pos%1000==0):
                    print(pos)
                    break
        except Exception as e:
            pass
        sentence=sentence+words
except Exception as e:
    print(e)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000


In [37]:
len(sentence)

266039

In [38]:
model=word2vec.Word2Vec(sentence,min_count=3, size=100, window=5, workers=4)

In [44]:
def get_emotion(emotion,layer1,layer2,re):
    dit = {}
    try:
        dict_emotion={emotion:1}
        for i in model.wv.most_similar(emotion,topn=layer1): 
            dict_emotion[i[0]]=dict_emotion.get(i[0],0)+i[1]
            for j in model.wv.most_similar(i[0],topn=layer2):
                dict_emotion[j[0]]=dict_emotion.get(j[0],0)+i[1]*j[1]
        sortlist=sorted(dict_emotion.items(),key = lambda x:x[1],reverse = True)[:re]
        
        for i in sortlist:
            dit[i[0]] = i[1]
        return dit,sortlist
    except Exception as e:
#         print(e)
        return dit,0


def getScore(poem,layer1,layer2,re):
#     emotions=['喜','怒','忧','思','悲','恐','惊']
    emotions=['悲','惧','乐','怒','思','喜','忧']
    type_probability={}
    for emotion in emotions:
        dit,temp=get_emotion(emotion,layer1,layer2,re)
        score=0
        for i in poem:
            score+=dit.get(i,0)
        score=score/len(poem)
        type_probability[emotion]=score
    type_probability.values()
#     print(type_probability)
    total_score=sum(list(type_probability.values()))
    if total_score==0:
        return '未知分类','未知分类'
    most =sorted(type_probability.items(),key = lambda x:x[1],reverse = True)[0:2]

    return most[0][0]


def distribution(poems,layer1=100,layer2=5,re=500):
    distribute_dict={}
    i=20000
    length = len(poems)
    for poem in poems[20000:21000]:
        i=i+1
        t=getScore(poem,layer1,layer2,re)
        
        distribute_dict[t]=distribute_dict.get(t,0)+1
#         distribute_dict[s]=distribute_dict.get(s,0)+1
        if i%100==0:
            print(str(i)+"       -------------"+str(i/length))
            
    return distribute_dict

In [45]:
def getPoems(dynasty):
    path = r'C:/Users/HP/新建文件夹/pa_chong/poem/朝代/'+dynasty+r'total.csv'
    f=open(path,'rb')
    content = pd.read_csv(f, header=None)[1:]
    f.close()
    poem=content[3]
    return poem

In [46]:
dynasty = '唐代'
poems = getPoems(dynasty)

In [47]:
fenbu = distribution(poems)

20100       -------------0.3979646385649514
20200       -------------0.3999445621399014
20300       -------------0.4019244857148514
20400       -------------0.4039044092898014
20500       -------------0.4058843328647514
20600       -------------0.4078642564397014
20700       -------------0.40984418001465145
20800       -------------0.4118241035896014
20900       -------------0.41380402716455145
21000       -------------0.4157839507395015


In [48]:
fenbu

{('未知分类', '未知分类'): 1,
 '乐': 39,
 '喜': 243,
 '忧': 7,
 '怒': 22,
 '思': 561,
 '悲': 117,
 '惧': 10}